# ERA Framework: Complete Genealogy + Bias Analysis

This notebook demonstrates the **complete ERA framework**:

1. **Graph-based genealogy tracking** - Map model evolution
2. **Three-level bias analysis** - L1/L2/L3 metrics for each model
3. **Lineage drift analysis** - How bias propagates through generations

## Example Scenario: Legal AI Model Family

We'll create a model family:
- GPT-3 (foundational)
  - → GPT-3-Legal (fine-tuned on legal texts)
    - → GPT-3-Criminal (fine-tuned on criminal law)
    - → GPT-3-Contract (fine-tuned on contracts)
  - ⇢ GPT-3-RAG (sibling with retrieval)

In [ ]:
# Install ERA framework
!pip install -q git+https://github.com/blacklotus1985/ERA-framework.git

In [ ]:
from era import (
    ModelGraph, 
    ModelNode, 
    RelationType,
    ERAAnalyzer,
    HuggingFaceWrapper
)
from era.graph_viz import visualize_graph, visualize_lineage
import matplotlib.pyplot as plt

## Step 1: Build Model Genealogy Graph

In [ ]:
# Create graph
graph = ModelGraph()

# Add foundational model
gpt3 = graph.add_model(
    model_id="gpt3",
    name="GPT-3",
    model_type="foundational",
    metadata={"size": "175B", "release": "2020"},
)

# Add fine-tuned models
gpt3_legal = graph.add_model(
    model_id="gpt3-legal",
    name="GPT-3 Legal",
    model_type="fine_tuned",
    metadata={"domain": "legal documents", "examples": "50k"},
)

gpt3_criminal = graph.add_model(
    model_id="gpt3-criminal",
    name="GPT-3 Criminal Law",
    model_type="fine_tuned",
    metadata={"domain": "criminal law", "examples": "20k"},
)

gpt3_contract = graph.add_model(
    model_id="gpt3-contract",
    name="GPT-3 Contracts",
    model_type="fine_tuned",
    metadata={"domain": "contract law", "examples": "30k"},
)

# Add architectural variant (sibling)
gpt3_rag = graph.add_model(
    model_id="gpt3-rag",
    name="GPT-3 RAG",
    model_type="architectural",
    metadata={"modification": "retrieval-augmented"},
)

# Add edges (relationships)
graph.add_edge(gpt3, gpt3_legal, RelationType.FINE_TUNING)
graph.add_edge(gpt3_legal, gpt3_criminal, RelationType.FINE_TUNING)
graph.add_edge(gpt3_legal, gpt3_contract, RelationType.FINE_TUNING)
graph.add_edge(gpt3, gpt3_rag, RelationType.SIBLING)

print(f"Graph created with {len(graph.nodes)} nodes and {len(graph.edges)} edges")

## Step 2: Run ERA Analysis on Each Model

For demonstration, we'll simulate metrics.
In production, you'd run actual L1/L2/L3 analysis.

In [ ]:
# Simulate ERA metrics for each model
# In reality, you'd run: analyzer.analyze(test_contexts, target_tokens)

metrics_data = {
    "gpt3": {
        "l1_mean_kl": 0.0,
        "l2_mean_kl": 0.0,
        "l3_mean_delta": 0.0,
        "alignment_score": 0.0,
    },
    "gpt3-legal": {
        "l1_mean_kl": 0.25,
        "l2_mean_kl": 0.89,
        "l3_mean_delta": 0.00012,
        "alignment_score": 7417,  # Moderate shallow alignment
    },
    "gpt3-criminal": {
        "l1_mean_kl": 0.42,
        "l2_mean_kl": 1.35,
        "l3_mean_delta": 0.000018,
        "alignment_score": 75000,  # Very shallow!
    },
    "gpt3-contract": {
        "l1_mean_kl": 0.31,
        "l2_mean_kl": 0.95,
        "l3_mean_delta": 0.00085,
        "alignment_score": 1118,  # Better depth
    },
    "gpt3-rag": {
        "l1_mean_kl": 0.18,
        "l2_mean_kl": 0.52,
        "l3_mean_delta": 0.00001,  # RAG doesn't change embeddings much
        "alignment_score": 52000,  # Shallow (architectural, not learning)
    },
}

# Update graph nodes with metrics
for model_id, metrics in metrics_data.items():
    graph.nodes[model_id].metrics = metrics

print("ERA metrics added to all models")

## Step 3: Visualize Complete Genealogy

In [ ]:
# Visualize graph colored by alignment score
visualize_graph(
    graph,
    layout="hierarchical",
    highlight_metric="alignment_score",
    output_path="model_genealogy.png"
)

## Step 4: Analyze Lineage Drift

How does alignment score change from GPT-3 → Legal → Criminal?

In [ ]:
# Get lineage for criminal law model
criminal_lineage = graph.get_lineage(gpt3_criminal)
print(f"Lineage: {' → '.join([m.name for m in criminal_lineage])}")

# Analyze alignment score drift
drift_analysis = graph.analyze_lineage_drift(gpt3_criminal, "alignment_score")

print("\nAlignment Score Evolution:")
for model, score in zip(drift_analysis["lineage"], drift_analysis["metric_values"]):
    print(f"  {model}: {score:.0f}")

print(f"\nTotal drift: {drift_analysis['total_drift']:.0f}")
print(f"Verdict: {'SHALLOW ALIGNMENT - DO NOT DEPLOY' if drift_analysis['metric_values'][-1] > 10000 else 'Acceptable depth'}")

In [ ]:
# Visualize lineage evolution
visualize_lineage(
    graph,
    gpt3_criminal,
    metric="alignment_score",
    output_path="criminal_lineage.png"
)

## Step 5: Compare Siblings

How does GPT-3 base compare to GPT-3-RAG?

In [ ]:
from era.graph_viz import visualize_metric_comparison

# Compare all children of GPT-3
gpt3_children = graph.get_children(gpt3)
print(f"GPT-3 has {len(gpt3_children)} direct descendants")

visualize_metric_comparison(
    graph,
    [gpt3] + gpt3_children,
    metric="alignment_score",
    output_path="gpt3_family_comparison.png"
)

## Step 6: Identify Problematic Models

Which models in the family have alignment score > 10,000?

In [ ]:
# Find all models with shallow alignment
shallow_threshold = 10000

problematic = []
for node in graph.nodes.values():
    score = node.metrics.get("alignment_score", 0)
    if score > shallow_threshold:
        problematic.append((node.name, score))

print(f"\n⚠️  Models with alignment score > {shallow_threshold}:")
for name, score in sorted(problematic, key=lambda x: x[1], reverse=True):
    print(f"  • {name}: {score:.0f} - EXTREMELY SHALLOW")

print(f"\n❌ {len(problematic)}/{len(graph.nodes)} models are NOT production-ready")

## Step 7: Save Graph for Later Analysis

In [ ]:
# Save graph to JSON
graph.save("legal_model_genealogy.json")

# Later, reload:
# loaded_graph = ModelGraph.load("legal_model_genealogy.json")

print("✓ Graph saved to legal_model_genealogy.json")

## Key Insights from This Analysis

1. **Shallow alignment amplifies with specialization**
   - GPT-3 Legal: 7,417 (moderate)
   - GPT-3 Criminal: 75,000 (extreme!) 
   - Verdict: Criminal law model is **parrot**, not genuine learning

2. **Architectural modifications don't improve depth**
   - GPT-3 RAG: 52,000 (very shallow)
   - RAG adds retrieval but doesn't fix shallow embeddings

3. **Contract model shows better learning**
   - Score: 1,118 (acceptable)
   - L3 drift much higher (0.00085 vs 0.000018)
   - Actually learned concepts, not just surface patterns

## Recommendations

- ✅ **Deploy:** GPT-3 Contracts (score < 2,000)
- ❌ **Do NOT deploy:** GPT-3 Criminal, GPT-3 RAG
- 🔄 **Deep retrain:** Criminal law model needs 2-3x more data + longer training

---

**For more examples:**
- Full documentation: https://github.com/blacklotus1985/ERA-framework
- Paper (arXiv): [Coming January 2025]